In [ ]:
import os

import numpy as np
from matplotlib import pyplot as plt
import h5py as h5

import specpy as sp

from pipeline2 import PipelineLevels, AcquisitionPipeline, DefaultNameHandler, HDF5RichData, HDF5DataStore
from pipeline2.taskgeneration import (AcquisitionTaskGenerator, JSONFileConfigLoader, DefaultFOVSettingsGenerator,
                                      AlreadyImagedFOVFilter,
                                      DefaultScanOffsetsSettingsGenerator, DefaultLocationKeeper,
                                      DefaultLocationRemover, DefaultStageOffsetsSettingsGenerator,
                                      SpiralOffsetGenerator, NewestSettingsSelector, NewestDataSelector,
                                      BoundingBoxLocationGrouper, DefaultScanModeSettingsGenerator,
                                      DefaultScanFieldSettingsGenerator)
from pipeline2.detection import (LegacySpotPairFinder, SimpleSingleChannelSpotDetector, SimpleNucleusMidplaneDetector,
                                 SimpleLegacyFocusHold, SimpleLocationRepeater)
from pipeline2.imspector import ImspectorConnection, get_current_stage_coords
from pipeline2.stoppingcriteria import TimedStoppingCriterion

from pipeline2.taskgeneration import StitchedNewestDataSelector

In [ ]:
base_path = 'C:/Users//RESOLFT/Desktop/auto-sir/'
sample_name = 'a163_sir_tc4237_'

save_path = os.path.join(base_path, sample_name)

if not os.path.exists(save_path):
    os.makedirs(save_path)

In [ ]:
# plotting params
%matplotlib inline
plt.rcParams['figure.figsize'] = [7, 7]

# imspector object
im = sp.Imspector()

# init levels and name handler
pll = PipelineLevels('overview', 'detail')
name_handler = DefaultNameHandler(save_path, pll)

# abuse name_handler to get h5 filename
# calling with empty index gives just the random hash
h5file_name = name_handler.get_path([]).replace('.msr', '.h5')

# open resulting h5 file in context manager
with h5.File(h5file_name, mode='a') as h5fd:
    data_store = HDF5DataStore(h5fd, pll)

    # init pipeline
    pl = (AcquisitionPipeline('auto-sir-pipeline-1')
            .withImspectorConnection(ImspectorConnection(im).withVerbose())
            .withPipelineLevels(pll)
            .withNameHandler(name_handler)
            .withAddedStoppingCondition(TimedStoppingCriterion(21 * 60 * 60))) # MAX TIME HERE!!! in seconds!!!
    
    # attach h5 data store
    pl.withDataStorage(data_store)
    

    # overview task generator: same settings every time
    atg_overview = (AcquisitionTaskGenerator(pll.overview, 
                    DefaultLocationRemover(
                        JSONFileConfigLoader(['C:/Users/RESOLFT/Desktop/config_json/20180731_sir_ov-100x.json'])
                    ),
                    DefaultStageOffsetsSettingsGenerator(SpiralOffsetGenerator().withFOV([4e-5, 4e-5]).withStart(get_current_stage_coords(im))),
                    DefaultScanModeSettingsGenerator(['xyz'], True),
                    DefaultFOVSettingsGenerator([[5e-5, 5e-5, 1e-5]], [[3e-7, 3e-7, 4e-7]]),
                    DefaultStageOffsetsSettingsGenerator(SimpleLegacyFocusHold(NewestDataSelector(pl, pll.overview)).withVerbose(True))
                )
                .withDelay(.4))

    # init detector in overview
    detector = (SimpleNucleusMidplaneDetector(StitchedNewestDataSelector(pl, pll.overview))
                .withVerbose()
                .withPlot()
                .withFilter({'area' : (200, 10000)})
                .withFOVExpansion(1.4)
               )

    # detail task generation: settings from last overview + new FOV + grouped detections
    # optionally repeat measurement to check reproducibility
    
    
    # NOTE: change settings here
    atg_detail = AcquisitionTaskGenerator(pll.detail,
                             DefaultLocationKeeper(NewestSettingsSelector(pl, pll.overview)),
                             DefaultLocationRemover(
                                 JSONFileConfigLoader(['C:/Users/RESOLFT/Desktop/config_json/20180731_sir_detail2d-100x.json'])
                             ),
                             DefaultScanFieldSettingsGenerator(detector, [3e-8, 3e-8, None]))

    # with 'none' pattern
    '''
    atg_detail = AcquisitionTaskGenerator(pll.detail,
                             DefaultLocationKeeper(NewestSettingsSelector(pl, pll.overview)),
                             DefaultLocationRemover(
                                 JSONFileConfigLoader(['C:/Users/RESOLFT/Desktop/config_json/20180712_edu_635p-detail2d-100x.json'] * 2,
                                                      ['C:/Users/RESOLFT/Desktop/config_json/20180712_imspector_left2d.json', 
                                                      'C:/Users/RESOLFT/Desktop/config_json/20180712_imspector_leftNONE.json' ], False)
                             ),
                             DefaultScanFieldSettingsGenerator(detector, [3e-8, 3e-8, None]))
    '''
    atg_detail.withDelay(.2)
    
    # add filter to ignore new bounding boxes that have intersection-over-union >0.5
    # with any already scanned bounding boxes
    flt = AlreadyImagedFOVFilter(pl, pll.detail, 0.5)
    atg_detail.withFilters(flt)

    # overview callback: re-add overview
    pl.withCallbackAtLevel(atg_overview, pll.overview)
    # overview callback 2: detect & do detail
    pl.withCallbackAtLevel(atg_detail, pll.overview)

    # call overview atg once to add first measurement to pipeline
    atg_overview(pl)

    # GO
    pl.run()